# A6.2 Tic Tac Toe

For this assignment you will run a number of comparisons between different neural networks trained through Q-learning to predict Q functions for Player X and for Player O in a simple Tic Tac Toe game.  

All but one simple function is provided, so your effort will be in choosing the parameters for the experiments.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas

In [2]:
class Optimizers():

    def __init__(self, all_weights):
        '''all_weights is a vector of all of a neural networks weights concatenated into a one-dimensional vector'''
        
        self.all_weights = all_weights

        # The following initializations are only used by adam.
        # Only initializing mt, vt, beta1t and beta2t here allows multiple calls to adam to handle training
        # with multiple subsets (batches) of training data.
        self.mt = np.zeros_like(all_weights)
        self.vt = np.zeros_like(all_weights)
        self.beta1 = 0.9
        self.beta2 = 0.999
        self.beta1t = 1  # was self.beta1
        self.beta2t = 1  # was self.beta2

        
    def sgd(self, error_f, gradient_f, fargs=[], n_epochs=100, learning_rate=0.001, error_convert_f=None):
        '''
error_f: function that requires X and T as arguments (given in fargs) and returns mean squared error.
gradient_f: function that requires X and T as arguments (in fargs) and returns gradient of mean squared error
            with respect to each weight.
error_convert_f: function that converts the standardized error from error_f to original T units.
        '''

        error_trace = []
        epochs_per_print = n_epochs // 10

        for epoch in range(n_epochs):

            error = error_f(*fargs)
            grad = gradient_f(*fargs)

            # Update all weights using -= to modify their values in-place.
            self.all_weights -= learning_rate * grad

            if error_convert_f:
                error = error_convert_f(error)
            error_trace.append(error)

        return error_trace

    def adam(self, error_f, gradient_f, fargs=[], n_epochs=100, learning_rate=0.001, error_convert_f=None):
        '''
error_f: function that requires X and T as arguments (given in fargs) and returns mean squared error.
gradient_f: function that requires X and T as arguments (in fargs) and returns gradient of mean squared error
            with respect to each weight.
error_convert_f: function that converts the standardized error from error_f to original T units.
        '''
        
        # adam = Adaptive Moment Estimation

        alpha = learning_rate  # learning rate called alpha in original paper on adam
        epsilon = 1e-8
        error_trace = []
        epochs_per_print = n_epochs // 10

        for epoch in range(n_epochs):

            error = error_f(*fargs)
            grad = gradient_f(*fargs)

            # Finish Adam implementation here by updating
            #   self.mt
            #   self.vt
            #   self.beta1t
            #   self.beta2t
            # and updating values of self.all_weights
            
            # mt = beta1 * mt + (1 - beta1) * gradE_W
            gradE_W = learning_rate * grad
            self.mt = self.beta1 * self.mt + (1 - self.beta1) * (gradE_W)
            
            # vt = beta2 * vt + (1 - beta2) * np.square(gradE_W)
            self.vt = self.beta2 * self.vt + (1 - self.beta2) * np.square(gradE_W)
            
            # beta1t *= beta1
            self.beta1t *= self.beta1
            
            # beta2t *= beta2
            self.beta2t *= self.beta2
            
            # mhat = mt / (1 - beta1)
            mhat = self.mt / (1 - self.beta1t)
            
            # vhat = vt / (1 - beta2)
            vhat = self.vt / (1 - self.beta2t)
            
            # w -= rho * mhat / (np.sqrt(vhat) + epsilon)
            # rho = learning rate?
            self.all_weights -= learning_rate * mhat / (np.sqrt(vhat) + epsilon)
        

            if error_convert_f:
                error = error_convert_f(error)
            error_trace.append(error)


        return error_trace

In [3]:
class NeuralNetwork():


    def __init__(self, n_inputs, n_hiddens_per_layer, n_outputs, activation_function="tanh"):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs

        # Set self.n_hiddens_per_layer to [] if argument is 0, [], or [0]
        if n_hiddens_per_layer == 0 or n_hiddens_per_layer == [] or n_hiddens_per_layer == [0]:
            self.n_hiddens_per_layer = []
        else:
            self.n_hiddens_per_layer = n_hiddens_per_layer

        # Initialize weights, by first building list of all weight matrix shapes.
        n_in = n_inputs
        shapes = []
        for nh in self.n_hiddens_per_layer:
            shapes.append((n_in + 1, nh))
            n_in = nh
        shapes.append((n_in + 1, n_outputs))

        # self.all_weights:  vector of all weights
        # self.Ws: list of weight matrices by layer
        self.all_weights, self.Ws = self.make_weights_and_views(shapes)

        # Define arrays to hold gradient values.
        # One array for each W array with same shape.
        self.all_gradients, self.dE_dWs = self.make_weights_and_views(shapes)

        self.trained = False
        self.total_epochs = 0
        self.error_trace = []
        self.Xmeans = None
        self.Xstds = None
        self.Tmeans = None
        self.Tstds = None
        self.activation_function = activation_function


    def make_weights_and_views(self, shapes):
        # vector of all weights built by horizontally stacking flatenned matrices
        # for each layer initialized with uniformly-distributed values.
        all_weights = np.hstack([np.random.uniform(size=shape).flat / np.sqrt(shape[0])
                                 for shape in shapes])
        # Build list of views by reshaping corresponding elements from vector of all weights
        # into correct shape for each layer.
        views = []
        start = 0
        for shape in shapes:
            size =shape[0] * shape[1]
            views.append(all_weights[start:start + size].reshape(shape))
            start += size
        return all_weights, views


    # Return string that shows how the constructor was called
    def __repr__(self):
        return f'NeuralNetwork({self.n_inputs}, {self.n_hiddens_per_layer}, {self.n_outputs})'


    # Return string that is more informative to the user about the state of this neural network.
    def __str__(self):
        if self.trained:
            return self.__repr__() + f' trained for {self.total_epochs} epochs, final training error {self.error_trace[-1]}'


    def train(self, X, T, n_epochs, learning_rate, method='sgd'):
        '''
train: 
  X: n_samples x n_inputs matrix of input samples, one per row
  T: n_samples x n_outputs matrix of target output values, one sample per row
  n_epochs: number of passes to take through all samples updating weights each pass
  learning_rate: factor controlling the step size of each update
  method: is either 'sgd' or 'adam'
        '''

        # Setup standardization parameters
        if self.Xmeans is None:
            self.Xmeans = X.mean(axis=0)
            self.Xstds = X.std(axis=0)
            self.Xstds[self.Xstds == 0] = 1  # So we don't divide by zero when standardizing
            self.Tmeans = T.mean(axis=0)
            self.Tstds = T.std(axis=0)
            
        # Standardize X and T
        X = (X - self.Xmeans) / self.Xstds
        T = (T - self.Tmeans) / self.Tstds

        # Instantiate Optimizers object by giving it vector of all weights
        optimizer = Optimizers(self.all_weights)

        # Define function to convert value from error_f into error in original T units.
        error_convert_f = lambda err: (np.sqrt(err) * self.Tstds)[0] # to scalar

        if method == 'sgd':

            for epoch in n_epochs:
                error_trace = optimizer.sgd(self.error_f, self.gradient_f,
                                            fargs=[X, T], n_epochs=epoch,
                                            learning_rate=learning_rate,
                                            error_convert_f=error_convert_f)

        elif method == 'adam':

            error_trace = optimizer.adam(self.error_f, self.gradient_f,
                                         fargs=[X, T], n_epochs=n_epochs,
                                         learning_rate=learning_rate,
                                         error_convert_f=error_convert_f)

        else:
            raise Exception("method must be 'sgd' or 'adam'")
        
        self.error_trace = error_trace

        # Return neural network object to allow applying other methods after training.
        #  Example:    Y = nnet.train(X, T, 100, 0.01).use(X)
        return self

    
    def relu(self, s):
        '''S is n_samples x n_units'''
        Y = s.copy()
        Y[Y < 0] = 0
        return Y

    def grad_relu(self, s):
        '''S is n_samples x n_units'''
        dY = s.copy()
        return (dY > 0).astype(int)



    def forward_pass(self, X):
        '''X assumed already standardized. Output returned as standardized.'''
        self.Ys = [X]
        for W in self.Ws[:-1]:
            
            if self.activation_function == 'tanh':
                self.Ys.append(np.tanh(self.Ys[-1] @ W[1:, :] + W[0:1, :]))
                
            if self.activation_function == 'relu':
                self.Ys.append(self.relu(self.Ys[-1] @ W[1:, :] + W[0:1, :]))
                
        last_W = self.Ws[-1]
        self.Ys.append(self.Ys[-1] @ last_W[1:, :] + last_W[0:1, :])
        return self.Ys

    # Function to be minimized by optimizer method, mean squared error
    def error_f(self, X, T):
        Ys = self.forward_pass(X)
        mean_sq_error = np.mean((T - Ys[-1]) ** 2)
        return mean_sq_error

    # Gradient of function to be minimized for use by optimizer method
    def gradient_f(self, X, T):
        '''Assumes forward_pass just called with layer outputs in self.Ys.'''
        error = T - self.Ys[-1]
        n_samples = X.shape[0]
        n_outputs = T.shape[1]
        delta = - error / (n_samples * n_outputs)
        n_layers = len(self.n_hiddens_per_layer) + 1
        # Step backwards through the layers to back-propagate the error (delta)
        for layeri in range(n_layers - 1, -1, -1):
            # gradient of all but bias weights
            self.dE_dWs[layeri][1:, :] = self.Ys[layeri].T @ delta
            # gradient of just the bias weights
            self.dE_dWs[layeri][0:1, :] = np.sum(delta, 0)
            
            # Back-propagate this layer's delta to previous layer
            if self.activation_function == 'tanh':
                delta = delta @ self.Ws[layeri][1:, :].T * (1 - self.Ys[layeri] ** 2)
            
            if self.activation_function == 'relu':
                delta = delta @ self.Ws[layeri][1:, :].T * (self.grad_relu(self.Ys[layeri]))
                
        return self.all_gradients

    def use(self, X):
        '''X assumed to not be standardized. Return the unstandardized prediction'''
        # Standardize X using Xmeans and Xstds in model
        X = (X - self.Xmeans) / self.Xstds
        
        '''
        #print(len(self.all_weights), len(X), len(self.Ws))
        # Predict output values using weights in model
        #X = np.insert(X,0,1,axis=1)
        # predict = X1 @ model['w']
        #predict = X @ self.Ws
        # Unstandardize the predicted output values using Tmeans and Tstds in model
        #Y = predict * self.Tstds + self.Tmeans
        # Return the unstandardized output values
        #return Y
        '''
        Y = self.forward_pass(X)
        T = Y[-1] * self.Tstds + self.Tmeans

        # Y[1:]
        return T
    
        

In [4]:
def initial_state():
    return np.array([0] * 9)

def next_state(s, a, marker):  # s is a board, and a is an index into the cells of the board, marker is 1 or -1
    s = s.copy()
    s[a] = 1 if marker == 'X' else -1
    return s

def reinforcement(s):
    if won('X', s):
        return 1
    if won('O', s):
        return -1
    return 0

def won(player, s):
    marker = 1 if player == 'X' else -1
    combos = np.array((0,1,2, 3,4,5, 6,7,8, 0,3,6, 1,4,7, 2,5,8, 0,4,8, 2,4,6))
    return np.any(np.all(marker == s[combos].reshape((-1, 3)), axis=1))

def draw(s):
    return sum(s == 0) == 0

def valid_actions(state):
    return np.where(state == 0)[0]

In [5]:
def stack_sa(s, a):
    return np.hstack((s, a)).reshape(1, -1)

def other_player(player):
    return 'X' if player == 'O' else 'O'

In [6]:
def epsilon_greedy(Qnet, state, epsilon):
    
    actions = valid_actions(state)
    
    if np.random.uniform() < epsilon:
        # Random Move
        action = np.random.choice(actions)
        
    else:
        # Greedy Move
        np.random.shuffle(actions)
        Qs = np.array([Qnet.use(stack_sa(state, a)) for a in actions])
        action = actions[np.argmax(Qs)]
        
    return action

In [7]:
def make_samples(Qnets, initial_state_f, next_state_f, reinforcement_f, epsilon):
    '''Run one game'''
    Samples = {'X': {'SA': [], 'R': [], 'Qn': []},
               'O': {'SA': [], 'R': [], 'Qn': []}}

    s = initial_state_f()
    player = 'X'

    while True:
        
        a = epsilon_greedy(Qnets[player], s, epsilon)
        sn = next_state_f(s, a, player)
        r = reinforcement_f(sn)

        Samples[player]['SA'].append(stack_sa(s, a))
        Samples[player]['R'].append(r)   # r is with respect to X
        Samples[player]['Qn'].append(0.0)  # fill in layer

        if r != 0 or draw(sn):
            Samples[other_player(player)]['R'][-1] = r  
            break

        s = sn
        player = other_player(player)  # switch

    for player in ['X', 'O']:
        Samps = Samples[player]
        Samps['SA'] = np.vstack(Samps['SA'])
        Samps['R'] = np.array(Samps['R']).reshape(-1, 1)
        Samps['Qn'] =  np.array(Samps['Qn']).reshape(-1 ,1)  # this statement added in A6.1

    # Assign all Qn's, based on following state, but go every other state to do all X values,
    ends_with_O = len(Samples['X']) > len(Samples['O'])
    if ends_with_O:
        # O wins
        Samples['X']['Qn'][:-1] = Qnets['X'].use(Samples['X']['SA'][1:, :])
        Samples['O']['Qn'][:-1] = Qnets['O'].use(Samples['O']['SA'][1:])
    else:
        # X wins or draw
        Samples['X']['Qn'][:-1] = Qnets['X'].use(Samples['X']['SA'][1:])
        Samples['O']['Qn'][:-1] = Qnets['O'].use(Samples['O']['SA'][1:])

    for player in ['X', 'O']:
        Samps = Samples[player]
        Samps['Qn'] = np.array(Samps['Qn']).reshape(-1, 1)

    return Samples

In [8]:
def plot_status(outcomes, epsilons, n_trials, trial):
    if trial == 0:
        return
    outcomes = np.array(outcomes)
    n_per = 10
    n_bins = (trial + 1) // n_per
    if n_bins == 0:
        return
    outcome_rows = outcomes[:n_per * n_bins].reshape((-1, n_per))
    outcome_rows = outcome_rows[:trial // n_per + 1, :]
    avgs = np.mean(outcome_rows, axis=1)
    
    plt.subplot(3, 1, 1)
    xs = np.linspace(n_per, n_per * n_bins, len(avgs))
    plt.plot(xs, avgs)
    plt.ylim(-1.1, 1.1)
    plt.xlabel('Games')
    plt.ylabel('Mean of Outcomes') # \n(0=draw, 1=X win, -1=O win)')
    plt.title(f'Bins of {n_per:d} Games')
    
    plt.subplot(3, 1, 2)
    plt.plot(xs, np.sum(outcome_rows == -1, axis=1), 'r-', label='Losses')
    plt.plot(xs, np.sum(outcome_rows == 0, axis=1), 'b-', label='Draws')
    plt.plot(xs, np.sum(outcome_rows == 1, axis=1), 'g-', label='Wins')
    plt.legend(loc='center')
    plt.ylabel(f'Number of Games\nin Bins of {n_per:d}')
    
    plt.subplot(3, 1, 3)
    plt.plot(epsilons[:trial])
    plt.ylabel('$\epsilon$')

In [9]:
def setup_standardization(Qnet, Xmeans, Xstds, Tmeans, Tstds):
    Qnet.Xmeans = np.array(Xmeans)
    Qnet.Xstds = np.array(Xstds)
    Qnet.Tmeans = np.array(Tmeans)
    Qnet.Tstds = np.array(Tstds)

In [10]:
from IPython.display import display, clear_output

def run(X_hidden_units_list_of_lists, O_hidden_units_list_of_lists, n_epochs_list, learning_rate_list, 
        repetitions=5, graphics=False):
    
    if graphics:
        fig = plt.figure(figsize=(10, 10))
        
    n_trials = 8000         # number of repetitions of makeSamples-updateQ loop
    
    gamma = 1.0        # discount factor
    final_epsilon = 0.01 # value of epsilon at end of simulation. Decay rate is calculated
    epsilon_decay = np.exp(np.log(final_epsilon) / (n_trials)) # to produce this final value

    results = []
    for n_epochs in n_epochs_list:
        for learning_rate in learning_rate_list:
            for X_nh in X_hidden_units_list_of_lists:
                for O_nh in O_hidden_units_list_of_lists:
                
                    last_fifth_outcomes = []

                    # RRn multiple experiments for these parameter values and average the results
                    for rep in range(repetitions):
                        
                        print(rep + 1, end=' ')
                        # Qnet for Player 'X'
                        QnetX = NeuralNetwork(9 + 1, X_nh, 1)
                        # Qnet for Player 'O'
                        QnetO = NeuralNetwork(9 + 1, O_nh, 1)
                        Qnets = {'X': QnetX, 'O': QnetO}

                        # Inputs are 9 TTT cells plus 1 action
                        setup_standardization(QnetX, [0] * 10, [1] * 10, [0], [1])
                        setup_standardization(QnetO, [0] * 10, [1] * 10, [0], [1])

                        epsilon = 1         # initial epsilon value
                        outcomes = []
                        epsilon_trace = []

                        # Train for n_trials
                        for trial in range(n_trials):

                            Samples = make_samples(Qnets, initial_state, next_state, reinforcement, epsilon)

                            Samps = Samples['X']
                            SA = Samps['SA']
                            R = Samps['R']
                            Qn = Samps['Qn']
                            T = R + gamma * Qn
                            Qnets['X'].train(SA, T, n_epochs, learning_rate, method='sgd')

                            Samps = Samples['O']
                            SA = Samps['SA']
                            R = - Samps['R']  # r is with respect to X, so negate it
                            Qn = Samps['Qn']
                            T = R + gamma * Qn
                            Qnets['O'].train(SA, T, n_epochs, learning_rate, method='sgd')

                            outcomes.append(Samples['X']['R'][-1])
                            epsilon_trace.append(epsilon)

                            epsilon *= epsilon_decay
                            
                            if graphics and (trial + 1 == n_trials or trial % (n_trials / 20) == 0):
                                plt.clf()
                                plot_status(outcomes, epsilon_trace, n_trials, trial)
                                clear_output(wait=True)
                                display(fig)

                        # For each repetition collect the mean of the outcome for the final fifth games
                        last_fifth_outcomes.append(np.mean(outcomes[-n_trials // 5:]))
                        print(f'{last_fifth_outcomes[-1]:.1f},', end=' ')
                        
                    results.append([X_nh, O_nh, n_epochs, learning_rate, np.mean(last_fifth_outcomes)])
                    print(results[-1])
                    
    if graphics:
        clear_output(wait=True)
        
    return pandas.DataFrame(results, columns=('X_nh', 'O_nh', 'n_epochs', 'lr', 'last_fifth_outcomes')), Qnets

## Strategies for X winning

Copy a version of `myresult()` into next cell that contains a network architecture with which X can win the game.  If you cannot get X to win the game, put one of the strategies you tried here.

In [ ]:
def myresult1():
    result, Qnets = run(X_hidden_units_list_of_lists=[[], [10, 10], [10,20]],
                        O_hidden_units_list_of_lists=[[], [100, 20], [20, 20, 20]],
                        n_epochs_list=[[5],[10],[20]],
                        learning_rate_list=[[0.001],[0.01],[0.1]], 
                        repetitions=5, graphics=False)
    return result



import time
start_time1 = time.time()

result1 = myresult1()

print(f'Took {(time.time() - start_time1) / 60.0:.1f} minutes.')

result1

1 0.9, 2 0.9, 3 0.9, 4 0.9, 5 0.9, [[], [], [5], [0.001], 0.9404999999999999]
1 0.9, 2 1.0, 3 1.0, 4 0.9, 5 0.9, [[], [100, 20], [5], [0.001], 0.947625]
1 0.9, 2 0.9, 3 0.9, 4 0.9, 5 0.9, [[], [20, 20, 20], [5], [0.001], 0.9390000000000001]
1 0.8, 2 0.9, 3 0.9, 4 0.9, 5 0.9, [[10, 10], [], [5], [0.001], 0.90425]
1 0.9, 2 1.0, 3 0.9, 4 0.9, 5 1.0, [[10, 10], [100, 20], [5], [0.001], 0.94575]
1 0.9, 2 0.8, 3 0.9, 4 0.9, 5 0.9, [[10, 10], [20, 20, 20], [5], [0.001], 0.9094999999999999]
1 1.0, 2 0.9, 3 0.9, 4 1.0, 5 0.9, [[10, 20], [], [5], [0.001], 0.93675]
1 1.0, 2 0.9, 3 0.9, 4 1.0, 5 1.0, [[10, 20], [100, 20], [5], [0.001], 0.9525]
1 0.9, 2 0.9, 3 0.9, 4 1.0, 5 0.9, [[10, 20], [20, 20, 20], [5], [0.001], 0.94825]
1 0.9, 2 1.0, 3 0.9, 4 0.9, 5 0.9, [[], [], [5], [0.01], 0.9400000000000001]
1 1.0, 2 0.9, 3 0.9, 4 0.9, 5 0.9, [[], [100, 20], [5], [0.01], 0.9337499999999999]
1 0.9, 2 0.9, 3 0.9, 4 0.6, 5 0.9, [[], [20, 20, 20], [5], [0.01], 0.8463750000000001]
1 1.0, 2 0.9, 3 0.9, 4 1.0, 5

<ipython-input-3-90d984758462>:171: RuntimeWarning: overflow encountered in multiply
  delta = delta @ self.Ws[layeri][1:, :].T * (1 - self.Ys[layeri] ** 2)
<ipython-input-3-90d984758462>:151: RuntimeWarning: overflow encountered in square
  mean_sq_error = np.mean((T - Ys[-1]) ** 2)
<ipython-input-3-90d984758462>:171: RuntimeWarning: overflow encountered in matmul
  delta = delta @ self.Ws[layeri][1:, :].T * (1 - self.Ys[layeri] ** 2)
<ipython-input-3-90d984758462>:171: RuntimeWarning: invalid value encountered in multiply
  delta = delta @ self.Ws[layeri][1:, :].T * (1 - self.Ys[layeri] ** 2)
<ipython-input-3-90d984758462>:165: RuntimeWarning: overflow encountered in matmul
  self.dE_dWs[layeri][1:, :] = self.Ys[layeri].T @ delta
<ipython-input-3-90d984758462>:165: RuntimeWarning: invalid value encountered in matmul
  self.dE_dWs[layeri][1:, :] = self.Ys[layeri].T @ delta
<ipython-input-2-4c41ffc0fa23>:36: RuntimeWarning: invalid value encountered in subtract
  self.all_weights -= le

0.3, 2 

/usr/local/anaconda3/latest/lib/python3.8/site-packages/numpy/core/_methods.py:151: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
<ipython-input-3-90d984758462>:145: RuntimeWarning: invalid value encountered in matmul
  self.Ys.append(self.Ys[-1] @ last_W[1:, :] + last_W[0:1, :])
<ipython-input-3-90d984758462>:151: RuntimeWarning: invalid value encountered in subtract
  mean_sq_error = np.mean((T - Ys[-1]) ** 2)
<ipython-input-3-90d984758462>:157: RuntimeWarning: invalid value encountered in subtract
  error = T - self.Ys[-1]


0.3, 3 0.3, 4 0.3, 5 0.3, [[], [], [5], [0.1], 0.28625]
1 0.2, 2 0.1, 3 0.1, 4 0.1, 5 0.1, [[], [100, 20], [5], [0.1], 0.12137500000000001]
1 -0.0, 2 0.1, 3 -0.1, 4 0.1, 5 -0.0, [[], [20, 20, 20], [5], [0.1], 0.018000000000000002]
1 0.6, 2 0.6, 3 0.7, 4 0.7, 5 0.6, [[10, 10], [], [5], [0.1], 0.65225]
1 0.9, 2 0.9, 3 1.0, 4 0.8, 5 0.9, [[10, 10], [100, 20], [5], [0.1], 0.8943749999999999]
1 1.0, 2 0.9, 3 0.9, 4 0.7, 5 0.6, [[10, 10], [20, 20, 20], [5], [0.1], 0.8234999999999999]
1 0.7, 2 0.6, 3 0.6, 4 0.7, 5 0.6, [[10, 20], [], [5], [0.1], 0.627375]
1 0.8, 2 0.7, 3 0.9, 4 0.7, 5 0.9, [[10, 20], [100, 20], [5], [0.1], 0.80425]
1 0.7, 2 0.9, 3 0.7, 4 1.0, 5 0.9, [[10, 20], [20, 20, 20], [5], [0.1], 0.817]
1 0.9, 2 0.9, 3 0.9, 4 0.9, 5 0.9, [[], [], [10], [0.001], 0.9355]
1 0.2, 2 0.4, 3 0.7, 4 0.9, 5 0.2, [[], [100, 20], [10], [0.001], 0.492375]
1 0.9, 2 0.9, 3 0.9, 4 0.8, 5 0.9, [[], [20, 20, 20], [10], [0.001], 0.9101250000000001]
1 1.0, 2 0.9, 3 0.9, 4 0.9, 5 1.0, [[10, 10], [], [10], 

In [ ]:
def myresult2():
    result, Qnets = run(X_hidden_units_list_of_lists=[[10], [10], [10]],
                        O_hidden_units_list_of_lists=[[], [100, 10], [20,10]],
                        n_epochs_list=[[5],[15],[25]],
                        learning_rate_list=[[0.01],[0.001],[0.1]], 
                        repetitions=5, graphics=False)
    return result



start_time2 = time.time()

result2 = myresult2()

print(f'Took {(time.time() - start_time2) / 60.0:.1f} minutes.')

result2

In [ ]:
def myresult3():
    result, Qnets = run(X_hidden_units_list_of_lists=[[], [100, 10], [5]],
                        O_hidden_units_list_of_lists=[[], [10,100], [50,10]],
                        n_epochs_list=[[3],[6],[9]],
                        learning_rate_list=[[0.1],[0.001],[0.01]], 
                        repetitions=5, graphics=False)
    return result



start_time3 = time.time()

result3 = myresult3()

print(f'Took {(time.time() - start_time3) / 60.0:.1f} minutes.')

result3

I could not get X to win the game. Partly due to the expirements taking very long to complete. Also, my computer would crash at times. Therefore, it was difficult to gather enough information to get the job done correctly. The issue might've been selecting the size of each layer for the four hyperparameters to be too large. Also, the implementaion of NeuralNetworks need to be tweaked to better aid the expirments. The tweaks however aren't very clear.

## Strategies for O winning

Copy a version of `myresult()` into next cell that contains a network architecture with which O can win the game.  If you cannot get O to win the game, put one of the strategies you tried here.

In [ ]:
def myresult4():
    result, Qnets = run(X_hidden_units_list_of_lists=[[], [100, 40], [40, 40, 40]],
                        O_hidden_units_list_of_lists=[[], [10, 10], [10,20]],
                        n_epochs_list=[[5],[10],[15]],
                        learning_rate_list=[[0.001],[0.01],[0.1]], 
                        repetitions=5, graphics=False)
    return result


start_time4 = time.time()

result4 = myresult4()

print(f'Took {(time.time() - start_time4) / 60.0:.1f} minutes.')

result4

In [ ]:
def myresult5():
    result, Qnets = run(X_hidden_units_list_of_lists=[[], [100, 10], [20,10]],
                        O_hidden_units_list_of_lists=[[100], [100], [100]],
                        n_epochs_list=[[3],[6],[9]],
                        learning_rate_list=[[0.01],[0.0001],[0.1]], 
                        repetitions=5, graphics=False)
    return result


start_time5 = time.time()

result5 = myresult5()

print(f'Took {(time.time() - start_time5) / 60.0:.1f} minutes.')

result5

In [ ]:
def myresult6():
    result, Qnets = run(X_hidden_units_list_of_lists=[[], [10,50], [50,10]],
                        O_hidden_units_list_of_lists=[[], [100, 10], [50]],
                        n_epochs_list=[[10],[20],[30]],
                        learning_rate_list=[[0.1],[0.001],[0.0001]], 
                        repetitions=5, graphics=False)
    return result


start_time6 = time.time()

result6 = myresult6()

print(f'Took {(time.time() - start_time6) / 60.0:.1f} minutes.')

result6

I could not get O to win the game because I had the epoch numbers too large when I was running my expirements. Thus it lead to extremely slow completion times. I learne my mistakes and corrected them. Then, things ran a lot faster but not fast enough so the number of expirements I could run was very minimal thus leading to an inability to properly get O to win the game. There maybe exist a possiblity that I also implemented my Neural Networks incorrectly.

## Grading and Check-in

*There is no grader script for this assignment.*  The entirety of your grade is based on trying the required number of experiments and presenting your results.

## Extra Credit

For 1 point of extra credit do the following steps.

1. Call `run` using your best parameter values and for 1 repetition.
2. Create four boards for which it is `X`'s turn. Using the returned `Qnets` print a display of the Q values generated by `Qnets['X']` in a 3 x 3 table corresponding to the tic tac toe board, for each of these four boards.
3. Create four boards for which it is `O`'s turn. Using the returned `Qnets` print a display of the Q values generated by `Qnets['O']` in a 3 x 3 table corresponding to the tic tac toe board, for these four boards.
4. Discuss the values.  Do they make sense?